In [ ]:
!python -m pip install paddlepaddle-gpu==2.4.1.post112 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!git clone https://github.com/PaddlePaddle/PaddleOCR.git
%cd /content/PaddleOCR
!pip install -r requirements.txt
!pip install --upgrade paddleocr
!pip install fpdf
!pip install langdetect

In [ ]:
from paddleocr import PaddleOCR
import matplotlib.pyplot as plt
import cv2

# Initialize PaddleOCR with the Farsi language model
fa_ocr = PaddleOCR(use_angle_cls=True, lang='fa', use_gpu=False)

en_ocr = PaddleOCR(use_angle_cls=True, lang='en', use_gpu=False)

# Read the image containing the Farsi text
img_path = '/out/p1.jpg'

image = cv2.imread(img_path)


# Perform OCR on the image
result = fa_ocr.ocr(img_path, cls=True)

en_results = en_ocr.ocr(img_path,cls=True)


print(en_results)

print(result)

# Print the length of the result list
print("Number of text boxes detected:", len(*result))

In [ ]:
def text_accur(text_boxs):
  ret = []
  #print(text_boxs)
  for text in text_boxs:
    accur = text[1][1]
    #print(text , accur)
    if(accur>0.92):
      ret.append(text)
  return ret

In [ ]:
result[0] = text_accur(result[0])

In [ ]:
def merge_sort(results,avg):
    if len(results) <= 1:
        return results

    # Divide the list into two halves
    mid = len(results) // 2
    left_half = results[:mid]
    right_half = results[mid:]

    # Recursively sort each half
    left_sorted = merge_sort(left_half,avg)
    right_sorted = merge_sort(right_half,avg)

    # Merge the sorted halves
    merged = []
    left_index = 0
    right_index = 0
    while left_index < len(left_sorted) and right_index < len(right_sorted):
        # Sort by right-to-left order
        if abs(left_sorted[left_index][0][4][1] - right_sorted[right_index][0][4][1]) < avg/4:
          if left_sorted[left_index][0][4][0] < right_sorted[right_index][0][4][0] :
             merged.append(right_sorted[right_index])
             right_index += 1
          else:
              merged.append(left_sorted[left_index])
              left_index += 1
        else:
            if left_sorted[left_index][0][4][1] > right_sorted[right_index][0][4][1] :
              merged.append(right_sorted[right_index])
              right_index += 1
            else:
              merged.append(left_sorted[left_index])
              left_index += 1

    # Append the remaining elements (if any)
    merged += left_sorted[left_index:]
    merged += right_sorted[right_index:]

    return merged

In [ ]:
def sorting(word_list):
  import statistics
  avg=0
  num=[]
  for item in word_list[0]:
      corners, text = item
      center_x = sum([corner[0] for corner in corners]) / 4
      center_y = sum([corner[1] for corner in corners]) / 4
      center = [center_x, center_y]
      num.append((corners[2][1]+corners[3][1])-(corners[0][1] +corners[1][1]))
      corners.append(center)
  avg = average = statistics.mean(num)
  return (avg,merge_sort(*word_list,avg))

In [ ]:
#sorting
print(result)
avg,result=sorting(result)
print(result)

In [ ]:
Total_text=''
line =0
for item in result:
    corners, text = item
    if abs(line - corners[4][1]) > avg/4:
      Total_text+='\n'
    line = corners[4][1]
    Total_text+=text[0][-1::-1]+' '
    x1, y1 = int(corners[0][0]), int(corners[0][1])
    x2, y2 = int(corners[2][0]), int(corners[2][1])
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
print(Total_text)
# Display the image with bounding boxes
plt.imshow(image)
plt.show()
plt.save('result.jpg')

In [ ]:
with open('/out/index.html', 'w') as f:
  text='''
  <!DOCTYPE html>
<html>
<head>
  <title>Page Title</title>
  <style>
    .rainbow-text {
      background: linear-gradient(to right, red, orange, yellow, green, blue, indigo, violet);
      -webkit-background-clip: text;
      -webkit-text-fill-color: transparent;
      text-align: center;
    }
  </style>
</head>
<body >

  <section onresize="updateFontSize()">
  '''
  text+=f'<img src="{img_path}">\n'
  for item in result:
    corners, content = item
    text+='<p class="rainbow-text"  onresize="updateFontSize()" style="position:absolute; overflow: auto; font-size: 30px;'
    text+= f'top:{corners[4][1]}px; left:{corners[4][0]}px;">{content[0][-1::-1]}</p>\n'
  text+='''
  </section>

</body>
</html>
  '''
  f.write(text)

In [ ]:
from fpdf import FPDF
import arabic_reshaper
import langdetect

from bidi.algorithm import get_display
class PDF(FPDF):
   pass

image_height, image_width, _ = image.shape
print(image_height, image_width , '\n\n')

pdf=PDF(orientation='P', unit='mm', format=(image_width*1.2, image_height*1.2))
pdf.add_font('DejaVu', '', '/out/font.ttf', uni=True)

pdf.add_page()
pdf.set_font('DejaVu', '', avg/2)

for item in result:
    corners, text = item
    try:
      print(text[0] , langdetect.detect(text[0]))

      if langdetect.detect(text[0])=='ar' or langdetect.detect(text[0])=='fa':
        text=text[0][-1::-1]

        x, y = corners[4]

        pdf.set_xy(x, y)

        text = arabic_reshaper.reshape(text)

        pdf.cell(200, 10, txt=get_display(text), ln=1, align="C")
      else:
        text=text[0]
        x, y = corners[4]
        pdf.set_xy(x, y)
        pdf.cell(200, 10, txt=text, ln=1, align="C")
    except:
      text=text[0]
      x, y = corners[4]
      pdf.set_xy(x, y)
      pdf.cell(200, 10, txt=text, ln=1, align="C")
pdf.output('/out/index.pdf')